In [120]:
import os
import geocoder
import pandas as pd
import folium

In [121]:
main_dir = os.getcwd()
data_dir = os.path.join(main_dir, 'data')

In [122]:
def add_xy_to_csv(infile, location_header, outfile):
    df = pd.read_csv(os.path.join(data_dir, infile), encoding='utf8')
    locations = df[location_header].tolist()
    geocoded = []
    for location in locations:
        loc = geocoder.mapbox(location, key='pk.eyJ1IjoiZ2lzcHJhY3RpY2UiLCJhIjoiY2tnbXN2dGhyMGZ2ajJ0cGcxbnJ6ZGpyeiJ9.JjUCzWfOI5Oju7YxIrRR2A')
        geocoded.append(loc.json)
    
    temp_list = []
    for row in geocoded:
        temp_list.append(row['lng'])
    df.insert(0, 'longitude', temp_list, True)
    temp_list = []
    for row in geocoded:
        temp_list.append(row['lat'])
    df.insert(1, 'latitude', temp_list, True)
    df.to_csv(os.path.join(data_dir, outfile))

In [123]:
def write_to_map(csv, location_header, color):
    df = pd.read_csv(os.path.join(data_dir, csv))
    for i in range(0,len(df)):
        folium.CircleMarker(
            [df.iloc[i]['latitude'], df.iloc[i]['longitude']],
            popup='<div width=100%><strong>' + df.iloc[i]['Facility'] + '</strong><br>' + df.iloc[i][location_header] + '</div>',
            fill=color,
            color=color,
            opacity=0.75
            ).add_to(map_vac_loc)

In [125]:
infile, outfile = 'vac_loc.csv', 'vac_loc_geocoded.csv'
location_header = 'Address'

In [132]:
add_xy_to_csv('vac_loc.csv', 'Address', 'vac_loc_geocoded.csv')
#######
#####
####
###
##
# Note that Bernice Pharmacy keeps getting geocoded incorrectly must be hand coded in: 
# (32.8218659864738, -92.65657396931203)

In [135]:
map_vac_loc = map = folium.Map(location=[30.9843, -91.9623], 
                               tiles='cartodb positron', 
                               zoom_start=7
                              )
write_to_map(outfile, location_header, '#7AA07F')

In [136]:
map_vac_loc

In [129]:
# map_vac_loc.save('vac_loc.html')

In [137]:
# Create GeoJSON for vaccination locations data
df = pd.read_csv(os.path.join(data_dir, outfile))
n = '{\n  "type": "FeatureCollection",\n  "features": ['
for i in range(0,len(df)):
    n += '''  
    {
      "type": "Feature",
      "geometry": {
        "type": "Point",
        "coordinates": [''' + str(df.iloc[i]['longitude']) + ',' + str(df.iloc[i]['latitude']) + ''']
      },
      "properties": {
        "Parish": "''' + str(df.iloc[i]['Parish']) + '''",
        "Facility": "''' + str(df.iloc[i]['Facility']) + '''",
        "Address": "''' + str(df.iloc[i]['Address']) + '''",
        "Website": "''' + str(df.iloc[i]['Website']) + '''",
        "Phone": "''' + str(df.iloc[i]['Phone']) + '''"
      }
    },'''
n = n[:-1]    
n += '\n  ]\n}'        

with open(os.path.join(data_dir, 'vac_loc.geojson'), 'w') as file:
    file.write(n)

In [138]:
# Add census tract data to census tract shapefile and export to geoJSON

import geopandas as gpd

# Convert census tract TIGER/Line shapefile to GeoJSON
tracts_shp = os.path.join(data_dir, 'tl_2019_22_tract/tl_2019_22_tract.shp')
tracts_shp_df = gpd.read_file(tracts_shp)

cleaned = tracts_shp_df.drop(['STATEFP', 'COUNTYFP', 'TRACTCE', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'], axis=1)

cleaned.to_file("data/census_tracts_cleaned.geojson", driver='GeoJSON')

# Add vehicle access data to census tracts
tracts_df = gpd.read_file(os.path.join(data_dir, 'census_tracts.geojson'))

# Add full census tract code to tracts
geoid = []
for tract in tracts_df['GEOID']:
    geoid.append('1400000US' + tract)
tracts_df['GEO_ID'] = geoid
cleaned_tracts_df = tracts_df.drop(['GEOID'], axis=1)

# Merge ACS data to tracts data using census tracts as key
acs_data = os.path.join(data_dir, 'ACSDT5Y2019.B08201_2021-02-01T153625/cleaned.csv')
acs_df = pd.read_csv(acs_data)
merged_df = cleaned_tracts_df.merge(acs_df, left_on='GEO_ID', right_on='GEO_ID', how='left')
merged_df.to_file(os.path.join(data_dir, 'tracts_no_vehicle.geojson'), driver='GeoJSON')